# <span style='color:dodgerBlue; font-weight:bold;'>Store Sales</span>

### <span style='color:aqua; font-weight:bold;'>Context: </span>

Forecasts aren’t just for meteorologists. Governments forecast economic growth. Scientists attempt to predict the future population. And businesses forecast product demand—a common task of professional data scientists. Forecasts are especially relevant to brick-and-mortar grocery stores, which must dance delicately with how much inventory to buy. Predict a little over, and grocers are stuck with overstocked, perishable goods. Guess a little under, and popular items quickly sell out, leading to lost revenue and upset customers. More accurate forecasting, thanks to machine learning, could help ensure retailers please customers by having just enough of the right products at the right time.

Current subjective forecasting methods for retail have little data to back them up and are unlikely to be automated. The problem becomes even more complex as retailers add new locations with unique needs, new products, ever-transitioning seasonal tastes, and unpredictable product marketing.


### <span style='color:aqua; font-weight:bold;'> Import Packages </span>

In [1]:
# Essentials
import numpy as np
import pandas as pd

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline

# Machine learning and statistical
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
#from xgboost import XGBRegressor

# Ignore useless warnings
import warnings
warnings.filterwarnings('ignore')

### <span style='color:aqua; font-weight:bold;'> Load the Data </span>

In [2]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

oil = pd.read_csv('Data/oil.csv')
stores = pd.read_csv('Data/stores.csv')
holidays = pd.read_csv('Data/holidays_events.csv')
transactions = pd.read_csv('Data/transactions.csv')

# <span style='color:dodgerblue; font-weight:bold;'> Exploratory Data Analysis</span> 

### <span style='color:aqua; font-weight:bold;'> Train </span>

In [3]:
train.shape

(3000888, 6)

In [4]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
train = train.drop('id', axis=1)

In [6]:
train.head(1)

,date,store_nbr,family,sales,onpromotion
0,2013-01-01,1,AUTOMOTIVE,0.0,0


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date         object 
 1   store_nbr    int64  
 2   family       object 
 3   sales        float64
 4   onpromotion  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 114.5+ MB


We have to change data type of "date" feature from object to date

In [8]:
train['date'] = pd.to_datetime(train['date'])

In [9]:
train.describe().T

,count,mean,min,25%,50%,75%,max,std
date,3000888,2015-04-24 08:27:04.703088384,2013-01-01 00:00:00,2014-02-26 18:00:00,2015-04-24 12:00:00,2016-06-19 06:00:00,2017-08-15 00:00:00,NaN
store_nbr,3000888.0,27.5,1.0,14.0,27.5,41.0,54.0,15.585787
sales,3000888.0,357.775749,0.0,0.0,11.0,195.84725,124717.0,1101.997721
onpromotion,3000888.0,2.60277,0.0,0.0,0.0,0.0,741.0,12.218882


In [10]:
train.isnull().sum()

date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [11]:
train.duplicated().sum()

0

### <span style='color:aqua; font-weight:bold;'> Holiday Events</span>

In [12]:
holidays.shape

(350, 6)

In [13]:
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [14]:
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [15]:
holidays['date'] = pd.to_datetime(holidays['date'])

In [16]:
holidays.describe().T

,count,mean,min,25%,50%,75%,max
date,350,2015-04-24 00:45:15.428571392,2012-03-02 00:00:00,2013-12-23 06:00:00,2015-06-08 00:00:00,2016-07-03 00:00:00,2017-12-26 00:00:00


In [17]:
holidays.isnull().sum()

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

In [18]:
holidays.duplicated().sum()

0

### <span style='color:aqua; font-weight:bold;'> Oil </span>

In [19]:
oil.shape

(1218, 2)

In [20]:
oil.head(5)

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [21]:
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [22]:
oil['date'] = pd.to_datetime(oil['date'])

In [23]:
oil.describe().T

,count,mean,min,25%,50%,75%,max,std
date,1218,2015-05-02 12:00:00,2013-01-01 00:00:00,2014-03-03 06:00:00,2015-05-02 12:00:00,2016-06-30 18:00:00,2017-08-31 00:00:00,NaN
dcoilwtico,1175.0,67.714366,26.19,46.405,53.19,95.66,110.62,25.630476


#### Handling Missing Values

In [24]:
oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

We will .............

In [25]:
# Handling numarical missing values using SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(oil[['dcoilwtico']])
oil[['dcoilwtico']]=imputer.transform(oil[['dcoilwtico']])

In [26]:
oil.isnull().sum().sum() # Checking if there is any missing values

0

In [27]:
oil.duplicated().sum()

0

### <span style='color:aqua; font-weight:bold;'> Stores </span>

In [28]:
stores.shape

(54, 5)

In [29]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [30]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [31]:
stores.isnull().sum()

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

In [32]:
stores.duplicated().sum()

0

### <span style='color:aqua; font-weight:bold;'> Test </span>

In [33]:
test.shape

(28512, 5)

In [34]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [35]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


In [36]:
test['date'] = pd.to_datetime(test['date'])

In [37]:
test.isnull().sum()

id             0
date           0
store_nbr      0
family         0
onpromotion    0
dtype: int64

In [38]:
test.duplicated().sum()

0

# <span style='color:dodgerblue; font-weight:bold;'>Plot relationship between variables </span> 

Top 10 Product Families by Sales

Top 10 Stores by Sales

Top 10 Stores by Number of Items on Promotion

Sales Trends Over Time (Top 10 Families)

Sales Distribution by (Top 10 Families)

Average Sales on Holidays vs Non-Holidays (Top 10 Families)

Oil Prices Over Time